目标：爬取豆瓣科幻电影数据，主要包括电影名称、类型、国家、上映日期、评分、评分人数等
目标网址：https://movie.douban.com/explore#!type=movie&tag=%E7%A7%91%E5%B9%BB&sort=time&page_limit=20&page_start=0

### 爬取动态网页的内容

动态网页的特点是：
1、浏览器上面你看到的数据，找不到与之对应的html标签。
      或者即使找到与之对应的标签，标签内部也没有数据
2、网址规律很难发现

爬动态网页的难点：
难以构建网址

之前我们爬取的网页，都是HTML静态生成的内容，直接从HTML源码中就能找到看到的数据和内容。
然而有一些网站的内容由前端的JavaScript动态生成，由于呈现在网页上的内容是由JS生成而来，我们能够在浏览器上看得到，但是在HTML源码中却发现不了。

遇到这种情况，我们应该如何对网页进行爬取呢？有两种方法：
1、从网页响应中找到JS脚本返回的JSON数据；2、使用Selenium对网页进行模拟访问

这里我们使用方法1：
我们可以找到JS调用的数据接口，从数据接口中找到网页中最后呈现的数据。
选择“网络”选项卡后，发现有很多响应，筛选只看XHR响应。（XHR是Ajax中的概念，表示XMLHTTPrequest）
通过预览就可以找到对应数据接口。

推荐阅读：https://www.jianshu.com/p/6e1591d696cd
https://my.oschina.net/u/4318033/blog/3335206/print

### 方法：Requests库 + lxml库（xpath语言） + Re正则表达式 

In [1]:
#coding:utf-8
import requests
import re,json
from  time import sleep
from lxml import etree  #选择用xpath解析
from fake_useragent import UserAgent                        
import csv
# import redis

In [2]:
#定义要爬取的内容，字典格式
def init_movie_list():
    dic_move_list = {
        '电影标识': '', '电影名': '', '链接': '', '类型': '', '国家': '', '上映日期': '', '年':'',
        '月': '', '日': '', '片长': '', '评分': '', '总评分人数': '', '短评数': '', '长评数': '',
#         '电影简介': '',
    }
    return dic_move_list

In [3]:
def get_list(sum):
#注意：这里我们需要获取的信息并不在网页html里
#     url = 'https://movie.douban.com/j/search_subjects?type=movie&tag=%E7%A7%91%E5%B9%BB&sort=time&page_limit=20&page_start={0}'.format(sum,)
    url = 'https://movie.douban.com/j/search_subjects?type=movie&tag=%E7%A7%91%E5%B9%BB&sort=time&page_limit=20&page_start='+ 'sum'  #两种方式都可以

#     headers={'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'zh-CN,zh;q=0.9', 'Connection': 'keep-alive', 'Cookie': 'll="108296"; bid=BbME6r_nuro; __yadk_uid=jdHi3QlBPfjdvh5YlbVdCR40Ypf2vvlA; _vwo_uuid_v2=D6726514AB365C52EFCDE704CAC845697|74724018438da00ae712a95e94cb5fc7; __utmc=30149280; __utmc=223695111; __utmz=30149280.1555418104.5.3.utmcsr=open.weixin.qq.com|utmccn=(referral)|utmcmd=referral|utmcct=/connect/qrconnect; push_noty_num=0; push_doumail_num=0; __utmv=30149280.18929; __utma=30149280.1273655765.1554811453.1555519339.1556175978.7; __utmb=30149280.1.10.1556175978; _pk_ref.100001.4cf6=%5B%22%22%2C%22%22%2C1556175980%2C%22https%3A%2F%2Fwww.douban.com%2F%22%5D; _pk_ses.100001.4cf6=*; __utma=223695111.385128923.1554811453.1555519339.1556175980.7; __utmb=223695111.0.10.1556175980; __utmz=223695111.1556175980.7.4.utmcsr=douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/; ap_v=0,6.0; dbcl2="189292246:a+SESTm2EuM"; ck=8mfX; _pk_id.100001.4cf6=c11972beecd6d174.1554811458.7.1556177322.1555519340.', 'Host': 'movie.douban.com', 'Referer': 'https://movie.douban.com/explore', 'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36', 'X-Requested-With': 'XMLHttpRequest'}
    headers = { "User-Agent": UserAgent(verify_ssl=False).random}     #伪装请求报头
    page = requests.get(url, headers=headers).text   #列表页得到的内容page为json格式
    
    sleep(3)   #控制爬虫爬取频率，更严格则可以用代理ip
    #print(url)
    #print(page)
    # 将字符串形式的json转换为dict字典
    page=json.loads(page)['subjects']
    #print(page)
    
    #提取数据，爬完一个电影的所有数据再爬下一个电影
    for i in page:          #for循环爬取这一页列表页上所有电影
        dic1 = init_movie_list()
        dic1['链接'] = i['url']       #这4个字段直接提取
        dic1['电影标识'] = i['id']
        dic1['评分'] = i['rate']
        dic1['电影名'] = i['title']
        get_detail(dic1)      #调用函数，获取详情页上的数据
        # break

In [4]:
def get_detail(dic1):   #获取详情页信息
    print('抓取电影信息')
#     headers = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'zh-CN,zh;q=0.9', 'Cache-Control': 'max-age=0', 'Connection': 'keep-alive', 'Cookie': 'll="108296"; bid=BbME6r_nuro; __yadk_uid=jdHi3QlBPfjdvh5YlbVdCR40Ypf2vvlA; _vwo_uuid_v2=D6726514AB365C52EFCDE704CAC845697|74724018438da00ae712a95e94cb5fc7; __utmc=30149280; __utmc=223695111; __utmz=30149280.1555418104.5.3.utmcsr=open.weixin.qq.com|utmccn=(referral)|utmcmd=referral|utmcct=/connect/qrconnect; push_noty_num=0; push_doumail_num=0; __utmv=30149280.18929; __utma=30149280.1273655765.1554811453.1555519339.1556175978.7; __utmb=30149280.1.10.1556175978; _pk_ref.100001.4cf6=%5B%22%22%2C%22%22%2C1556175980%2C%22https%3A%2F%2Fwww.douban.com%2F%22%5D; _pk_ses.100001.4cf6=*; __utma=223695111.385128923.1554811453.1555519339.1556175980.7; __utmb=223695111.0.10.1556175980; __utmz=223695111.1556175980.7.4.utmcsr=douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/; ap_v=0,6.0; dbcl2="189292246:a+SESTm2EuM"; ck=8mfX; _pk_id.100001.4cf6=c11972beecd6d174.1554811458.7.1556177658.1555519340.', 'Host': 'movie.douban.com', 'Referer': 'https://movie.douban.com/explore', 'Upgrade-Insecure-Requests': '1', 'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
    headers = { "User-Agent": UserAgent(verify_ssl=False).random}
    page = requests.get(url=dic1['链接'], headers=headers).text
    page2 = page
    # sleep(3)
  

    html = etree.HTML(page)  #解析爬取的网页page
    
    #片长的提取
    content = html.xpath('//*[@id="info"]/span[@property="v:runtime"]/@content') #content是list列表['']
    if content:
        print(content)
        dic1['片长'] = content[0]   #取第一个,它就变成str''
        
    #总评分人数的提取
    content = html.xpath('//*[@id="interest_sectl"]/div[1]/div[2]/div/div[2]/a/span/text()')
    if content:
        dic1['总评分人数'] = content[0]
    
    
    #电影类型的提取和转换     
    list1 = re.findall('<span class="pl">类型:</span>(.*?)<br/>', page)
#     print(list1)
    if list1:
        list2 = re.findall('<span property="v:genre">(.*?)</span>', list1[0])
        #print(list2)
        if list2:
            if len(list2) == 1:
#                 print("其它")
                dic1['类型'] = '其它'       #如果只有一个类型"科幻”，则类型为"它"
            else:
                for i in range(len(list2)):
                    if list2[i] != '科幻':
#                         print(list2[i])
                        dic1['类型'] = list2[i]    #否则取科幻以外的第一个类型
                        break

    #制片国家的提取和转换
    content = re.findall("制片国家/地区:</span> (.*?)<br/>", page)
    if content:                         #取第一个国家
        di_qu = content[0]
        c = di_qu.split('/')
        if c:
            dic1['国家'] = c[0]
#         print(c[0])

    page = page.replace('\\/', '/')
    page = page.replace('\n','').replace('\t','').replace('\r\n','').replace(' ','')
    page2 = page2.replace('\n','').replace('\t','').replace('\r\n','')

    #上映日期及年月日的提取和转换
    cate_list = re.findall('<spanclass="pl">上映日期:</span>(.*?)<br/>',page)
    if cate_list:
        c = re.findall('[0-9]{4}-[0-9]{2}-[0-9]{2}',cate_list[0])
        if c:
            dic1['上映日期'] = c[0]

            year = re.findall('[0-9]{4}',c[0])
            if year:
                dic1['年'] = year[0]
            month = re.findall('-([0-9]{2})-', c[0])
            if month:
                dic1['月'] = month[0]
            day = re.findall('-[0-9]{2}-([0-9]{2})', c[0])
            if day:
                dic1['日'] = day[0]

    #短评数和长评数的提取和转换
    c = re.findall('comments\?status=P">全部([0-9]*?)条</a>',page)
    if c:
        dic1['短评数'] = c[0]
    c = re.findall('<ahref="reviews">全部([0-9]*?)条</a>',page)
    if c:
        dic1['长评数'] = c[0]
        
        
    print(dic1)
    
    #向csv里存储数据
    writer.writerow([dic1['电影标识'],dic1['电影名'],dic1['链接'],dic1['类型'],dic1['国家'],
         dic1['上映日期'],dic1['年'],dic1['月'],dic1['日'],dic1['片长'],
         dic1['评分'],dic1['总评分人数'],
         dic1['短评数'],dic1['长评数']])

In [5]:
#创建一个包含列名的csv文件，用来存储爬下来的数据
# rd_cli = redis.Redis(db=6,encoding='utf-8')
#每次运行会清空原csv
name = '豆瓣电影.csv'
# csvfile = open(name,"w",newline="",encoding="utf-8-sig")
csvfile = open(name,"w",newline="",encoding="utf-8")
writer = csv.writer(csvfile)
#写入一行标题
writer.writerow(["电影标识" ,"电影名" ,"链接" ,"类型" ,"国家" ,"上映日期" ,"年", "月" ,"日" , "片长" ,"评分" , "总评分人数","短评数", "长评数"])

50

In [6]:
# csvfile.close()

In [7]:
#爬虫主程序
#翻页，每页20个电影，爬完这一页再爬下一页
for i in range(2):    #先爬取2页做演示
    print(i)
    num =i * 20
    print(num)
    get_list(num)    #调用函数，开始爬取
    
# get_list(60)
print("爬取完毕")

#关闭文件
csvfile.close()

0
0
抓取电影信息
['105']
{'电影标识': '34852765', '电影名': '卫星', '链接': 'https://movie.douban.com/subject/34852765/', '类型': '恐怖', '国家': '俄罗斯', '上映日期': '2020-04-23', '年': '2020', '月': '04', '日': '23', '片长': '105', '评分': '5.9', '总评分人数': '648', '短评数': '225', '长评数': '3'}
抓取电影信息
['74']
{'电影标识': '30373716', '电影名': '副本：义体置换', '链接': 'https://movie.douban.com/subject/30373716/', '类型': '动画', '国家': '美国', '上映日期': '2020-03-19', '年': '2020', '月': '03', '日': '19', '片长': '74', '评分': '6.8', '总评分人数': '4491', '短评数': '1379', '长评数': '17'}
抓取电影信息
['109']
{'电影标识': '4830483', '电影名': '喋血战士', '链接': 'https://movie.douban.com/subject/4830483/', '类型': '动作', '国家': '中国大陆 ', '上映日期': '2020-03-13', '年': '2020', '月': '03', '日': '13', '片长': '109', '评分': '5.6', '总评分人数': '11632', '短评数': '3387', '长评数': '36'}
抓取电影信息
['124']
{'电影标识': '2364086', '电影名': '隐形人', '链接': 'https://movie.douban.com/subject/2364086/', '类型': '惊悚', '国家': '澳大利亚 ', '上映日期': '2020-02-28', '年': '2020', '月': '02', '日': '28', '片长': '124', '评分': '7.3', '总评分人数': '117146', '短评

['128']
{'电影标识': '27109633', '电影名': '终结者：黑暗命运', '链接': 'https://movie.douban.com/subject/27109633/', '类型': '动作', '国家': '美国 ', '上映日期': '2019-11-01', '年': '2019', '月': '11', '日': '01', '片长': '128', '评分': '6.9', '总评分人数': '132901', '短评数': '40999', '长评数': '1466'}
抓取电影信息
['107']
{'电影标识': '34861255', '电影名': '无重力男子', '链接': 'https://movie.douban.com/subject/34861255/', '类型': '剧情', '国家': '意大利', '上映日期': '2019-10-21', '年': '2019', '月': '10', '日': '21', '片长': '107', '评分': '5.8', '总评分人数': '677', '短评数': '200', '长评数': '7'}
抓取电影信息
['98']
{'电影标识': '27066172', '电影名': '8号警报', '链接': 'https://movie.douban.com/subject/27066172/', '类型': '剧情', '国家': '加拿大', '上映日期': '2019-10-03', '年': '2019', '月': '10', '日': '03', '片长': '98', '评分': '5.7', '总评分人数': '4386', '短评数': '1051', '长评数': '9'}
抓取电影信息
['87']
{'电影标识': '26628382', '电影名': '小羊肖恩2：末日农场', '链接': 'https://movie.douban.com/subject/26628382/', '类型': '喜剧', '国家': '英国 ', '上映日期': '2019-12-28', '年': '2019', '月': '12', '日': '28', '片长': '87', '评分': '7.7', '总评分人数': '13909', '短

### 读取数据

In [8]:
import pandas as pd 
from pandas import DataFrame
# import numpy as np

In [9]:
data = pd.read_csv('豆瓣电影.csv')
# data = pd.read_excel('豆瓣电影.xls')
print('共爬取数据 %s 行（科幻电影个数）' %len(data))

# 查看每列数据的数据类型
print(data.dtypes)

共爬取数据 40 行（科幻电影个数）
电影标识       int64
电影名       object
链接        object
类型        object
国家        object
上映日期      object
年          int64
月          int64
日          int64
片长         int64
评分       float64
总评分人数      int64
短评数        int64
长评数        int64
dtype: object


In [10]:
data

,电影标识,电影名,链接,类型,国家,上映日期,年,月,日,片长,评分,总评分人数,短评数,长评数
0,34852765,卫星,https://movie.douban.com/subject/34852765/,恐怖,俄罗斯,2020-04-23,2020,4,23,105,5.9,648,225,3
1,30373716,副本：义体置换,https://movie.douban.com/subject/30373716/,动画,美国,2020-03-19,2020,3,19,74,6.8,4491,1379,17
2,4830483,喋血战士,https://movie.douban.com/subject/4830483/,动作,中国大陆,2020-03-13,2020,3,13,109,5.6,11632,3387,36
3,2364086,隐形人,https://movie.douban.com/subject/2364086/,惊悚,澳大利亚,2020-02-28,2020,2,28,124,7.3,117146,35216,440
4,34806231,假面骑士时王：盖茨王权,https://movie.douban.com/subject/34806231/,动作,日本,2020-02-28,2020,2,28,64,6.7,856,284,1
5,30264504,异界,https://movie.douban.com/subject/30264504/,其它,俄罗斯,2020-01-30,2020,1,30,111,6.3,12666,4102,59
6,26981442,深海异兽,https://movie.douban.com/subject/26981442/,剧情,美国,2020-01-07,2020,1,7,95,5.9,13418,4067,52
7,30242270,莫斯科陷落2,https://movie.douban.com/subject/30242270/,灾难,俄罗斯,2020-01-01,2020,1,1,129,5.7,1026,335,5
8,22265687,星球大战9：天行者崛起,https://movie.douban.com/subject/22265687/,动作,美国,2019-12-18,2019,12,18,142,6.3,70843,24762,576
9,30394535,被光抓走的人,https://movie.douban.com/subject/30394535/,剧情,中国大陆,2019-12-13,2019,12,13,131,6.9,142923,47092,1494
